# Identifying "Core" products at warehouse level

### Set weights of turn-and-earn (```w1```) & picks (```w2```) and skip to <a href='#skipto'>here</a>

In [51]:
w1 = 0.8
w2 = 0.2

### Begin formatting data (only use on 1st run)

In [20]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

df = pd.read_excel("../data/North Central Sales 6 Months GIT v2.xlsx", sheet_name="North Central")

df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('$', 'usd') for c in df.columns]

print(df.columns)

Index(['legacy_system_cd', 'legacy_division_cd', 'legacy_product_cd',
       'legacy_product_desc', 'core_item_flag', 'segment', 'PROD_CAT_1_NAME',
       'PROD_CAT_2_NAME', 'PROD_CAT_3_NAME', 'PROD_CAT_4_NAME',
       'legacy_vendor_cd', 'stocking_flag', 'LEGACY_CUSTOMER_CD',
       'saalfeld_customer_flag', 'national_acct_flag', 'ship-to_zip_code',
       'sales_channel', 'qty_6mos', 'cogs_6mos', 'Sales_6_mos', 'picks_6mos',
       'Margin_%', 'net_OH', 'net_OH_usd', 'pallet_quantity', 'item_poi_days',
       'DIOH'],
      dtype='object')


### Normalize ['qty_6mos', 'cogs_6mos', 'Sales_6mos', 'picks_6mos'] columns:

In [31]:
scaler = MinMaxScaler()

out = scaler.fit_transform(df[['qty_6mos', 'cogs_6mos', 'Sales_6_mos', 'picks_6mos']])

df2 = pd.DataFrame(out)
df2.columns = ['qty_6mos', 'cogs_6mos', 'Sales_6mos', 'picks_6mos']

for column in df2.columns:
    df[column] = df2[column]

Format: ```{Warehouse (Division): list(Unique products sold at division)}```

In [32]:
warehouses = df.legacy_division_cd.unique()
warehouse_to_prod = {}

for w in warehouses:
    warehouse_to_prod[w] = []

for w, p, sc in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.sales_channel):
    for ware in warehouses:
        # only add to list if sales channel == warehouse
        if w == ware and p not in warehouse_to_prod[w] and sc == "Warehouse":
            warehouse_to_prod[w].append(p)

# warehouse to number of individual different products stored            
for w in warehouses:
    print(w, len(warehouse_to_prod[w]))

19 2038
50 0
73 145
74 85
75 2493
77 1245
78 187
81 1785
82 1463
83 474
84 2800
85 1454
87 1229
89 2648
41 784


In [33]:
#Create a dictionary of warehouse & product -> total sales in a 6month period
wp_to_sales = {}
#Create a dictionary of warehouse & product -> total costs in a 6month period
wp_to_costs = {}
#Create a dictionary of warehouse & product -> total number of picks in a 6month period
wp_to_picks = {}
#Create a dictionary of warehouse & product -> total quantity sold in a 6month period
wp_to_quantity = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_sales[w,p] = []
        wp_to_costs[w,p] = []
        wp_to_picks[w,p] = []
        wp_to_quantity[w,p] = []

for w, p, s, c, pk, q in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.Sales_6_mos, df.cogs_6mos, df.picks_6mos, df.qty_6mos):
    if p in warehouse_to_prod[w]:
        wp_to_sales[w,p].append(s)
        wp_to_costs[w,p].append(c)
        wp_to_picks[w,p].append(pk)
        wp_to_quantity[w,p].append(q)

for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_sales[w,p] = sum(wp_to_sales[w,p])
        wp_to_costs[w,p] = sum(wp_to_costs[w,p])
        wp_to_picks[w,p] = sum(wp_to_picks[w,p])
        wp_to_quantity[w,p] = sum(wp_to_quantity[w,p])

#Create a dictionary of warehouse & product -> coreflag "Y" or "N"
wp_to_coreflag = {}
for w,p,cf in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.core_item_flag):
    if p in warehouse_to_prod[w]:
        if cf == "Y":
            wp_to_coreflag[w,p] = 1
        if cf == "N":
            wp_to_coreflag[w,p] = 0


#Create a dictionary of warehouse & product -> margin%
wp_to_margin = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        s = wp_to_sales[w,p]
        c = wp_to_costs[w,p]
        ##DATA HAS TO BE CLEANED SO THAT COSTS THAT ARE EQUAL TO 0 DO NOT EXIST
        if c == 0:
            wp_to_margin[w,p] = 0
        else:
            wp_to_margin[w,p] = 100*((s-c)/c)


In [34]:
#Create a Dictionary of a warehouse, product cd pair with its net OH in units, net OH $ as values, item poi days & DIOH
#Example:
#Warehouse 19 houses product 10012415 and it has 4 units in on hand inventory, $81.2 of on hand inventory,
#302.430555555556 of average poi in days & 38.5263664820611 of DIOH
#The dictionary will look like this: wp_to_oh[19, 10012415] = [4.0, 81.2, 302.430555555556, 38.5263664820611]
wp_to_stats = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_stats[w,p] = [0,0,0,0]

for w, p, oh_usd, oh, poi, DIOH in zip(df.legacy_division_cd.values, df.legacy_product_cd.values, df.net_OH_usd.values, df.net_OH.values,
                           df.item_poi_days.values, df.DIOH.values):
    if p in warehouse_to_prod[w]:
        if wp_to_stats[w,p] == [0,0,0,0]:
            wp_to_stats[w,p] = [oh, oh_usd, poi, DIOH]

In [35]:
#Create a Dictionary of warehouse & product to turn and earn
wp_to_TE = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        #netOH in quantity
        net_oh = wp_to_stats[w,p][0]
        #cost of goods sold
        cogs = wp_to_costs[w,p]
        #quantity sold
        q = wp_to_quantity[w,p]
        #margin percentage
        mp = wp_to_margin[w,p]
        
        #DATA MUST BE CLEANED THERE ARE 34 WAREHOUSE & PROD COMBINATIONS THAT 
        #PRODUCE A DENOMINATOR OF 0 IN THE TURN CALCULATION
        if 2*net_oh + q == 0:
            wp_to_TE[w,p] = 0
        else:
            turn = (2*cogs/(2*net_oh + q))
            wp_to_TE[w,p] = turn * mp    

### Rerun the following blocks to start using new weights: <a id='skipto'></a>

In [52]:
#Create a dictionary of warehouse & prod to score
wp_to_score = {}
for w in warehouses:
    for p in warehouse_to_prod[w]:
        wp_to_score[w,p] = w1*wp_to_TE[w,p]*w2*wp_to_picks[w,p]

In [53]:
#MODEL THAT ASSINGS cutoff% OF WORSE WAREHOUSE & PRODUCT COMBINATIONS IN THE NORTH CENTRAL REGION AS NON CORE
#sort the keys in ascending order by score
keys_by_score = sorted(wp_to_score, key=wp_to_score.__getitem__)

#input of percentage cut off for core item
#what bottom percentage of scores should we remove?
cutoff = 0.2

cutoffIdx = int(len(keys_by_score)*cutoff)
#the bottom cutoff% of the warehouse & product combination sorted by 
non_core = keys_by_score[:cutoffIdx]
core = keys_by_score[cutoffIdx:]

#Create our core flag rating dictionary
wp_to_ourcore = {}
for w,p in non_core:
    wp_to_ourcore[w,p] = 0

for w,p in core:
    wp_to_ourcore[w,p] = 1

#percentage similarity in our core flag and Veritiv
num_same = 0
for w in warehouses:
    for p in warehouse_to_prod[w]:
        if wp_to_ourcore[w,p] == wp_to_coreflag[w,p]:
            num_same += 1
similarity = num_same / len(wp_to_ourcore.values())
print("Percentage similarity: ", similarity*100)

Percentage similarity:  42.52257036643653


In [54]:
#MODEL THAT REMOVES cutoff% OF WORSE PRODUCTS AT THE WAREHOUSE LEVEL IN THE NORTH CENTRAL REGION AS NON CORE

#Create a dictionary of our core flag rating
wp_to_ourcore2 = {}
#input of cutoff point
cutoff = 0.2
#Sort each of the products in a warehouse by their score
for w in warehouses:
    prod_to_score = {}
    for p in warehouse_to_prod[w]:
        prod_to_score[p] = wp_to_score[w,p]
    #sort products inside of a warehouse by their score in ascending order
    prods_by_score = sorted(prod_to_score, key=prod_to_score.__getitem__)
    cutoffIdx = int(len(prods_by_score)*cutoff)
    #the bottom cutoff% of the warehouse & product combination sorted by 
    non_core_prods = prods_by_score[:cutoffIdx]
    core_prods = prods_by_score[cutoffIdx:]
    for p in non_core_prods:
        wp_to_ourcore2[w,p] = 0
    for p in core_prods:
        wp_to_ourcore2[w,p] = 1
#percentage similarity in our core flag and Veritiv
num_same = 0
for w in warehouses:
    for p in warehouse_to_prod[w]:
        if wp_to_ourcore2[w,p] == wp_to_coreflag[w,p]:
            num_same += 1
similarity = num_same / len(wp_to_ourcore2.values())
print("Percentage similarity: ", similarity*100)

Percentage similarity:  42.2145512480085
